In [1]:
from dotenv import find_dotenv,load_dotenv; _=load_dotenv(find_dotenv())

In [2]:
import json, os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
from pydantic import BaseModel, Field
from typing import Union, Any, Optional, Type
from enum import Enum
import googlemaps; gmaps = googlemaps.Client(key=os.environ['GOOGLE_MAPS_API_KEY'])

In [3]:
# gmaps.reverse_geocode(
#     latlng={"latitude": "35.1796","longitude": "136.9166"},
#     language='ja'
# )

In [4]:
import requests
def get_current_location():
    geo_request_url = 'https://get.geojs.io/v1/ip/geo.json'
    data = requests.get(geo_request_url).json()
    return {
        'latitude': data['latitude'],
        'longitude': data['longitude'],
    }
print(json.dumps(get_current_location(),indent=2,ensure_ascii=False))

get_current_location_tool_description = """\
You can get current location with no input.
"""

get_current_location_tool = StructuredTool.from_function(
    func = get_current_location,
    # name='CurrentLocationGetter',
    description=get_current_location_tool_description,
)

{
  "latitude": "35.1796",
  "longitude": "136.9166"
}


In [5]:
def simplified_place_dict(res):
    r = res[0]
    return {
        'address': r['formatted_address'],
        'location': r['geometry']['location'],
        'types': r['types']
    }

def find_place_with_text_func(query: str):
    places_result = gmaps.find_place(query,input_type='textquery')
    geocode_results = [
        gmaps.geocode(place_id=r['place_id'])
        for r in places_result['candidates']
    ]
    # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
    return [simplified_place_dict(r) for r in geocode_results]
# print(find_place_with_text_func(query='ガスト港東通店'))
find_place_tool_description = """\
A Find Place request takes a text input, and returns a place.
The text input can be any kind of Places data, for example,
a name, address, or phone number.
"""

find_place_tool = Tool.from_function(
    func = find_place_with_text_func,
    name='PlaceSearch',
    description=find_place_tool_description,
)

In [42]:
class Mode(Enum):
    DRIVING = "driving"
    WALKING = "walking"
    BYCYCLING = "bicycling"
    TRANSIT = "transit"

class DirectionsInput(BaseModel):
    # origin: Union[str,Any] = Field(
    origin: Union[str,dict] = Field(
        description='The address in string or latitude/longitude in dict from which you wish to calculate directions.'
    )
    # destination: Union[str,Any] = Field(
    destination: Union[str,dict] = Field(
        description='The address in string or latitude/longitude in dict from which you wish to calculate directions.'
    )
    mode: Mode = Field(
        description='Specifies the mode of transport to use when calculating directions.'
    )
    # departure_time: Optional[int] = Field(
    #     description='Specifies the desired time of departure. Note: you can't specify both departure_time and arrival_time.'
    # )
    # arrival_time: Optional[int] = Field(
    #     description='Specifies the desired time of arrival for transit directions. Note: you can't specify both departure_time and arrival_time.'
    # )

def simplified_directions_dict(res):
    r = res['legs'][0]
    return {
        'arrival_time': r['arrival_time']['text'] if 'arrival_time' in r.keys() else None,
        'departure_time': r['departure_time']['text'] if 'departure_time' in r.keys() else None,
        'distance': r['distance']['text'] if 'distance' in r.keys() else None,
        'duration': r['duration']['text'] if 'duration' in r.keys() else None,
        'steps': [
            {
                'distance': s['distance']['text'],
                'duration': s['duration']['text'],
                'travel_mode': s['travel_mode'],
                "instruction": s['html_instructions'],
                'transit_details': {
                    'arrival_stop': s['transit_details']['arrival_stop'],
                    'departure_stop': s['transit_details']['departure_stop'],

                } if 'transit_details' in s.keys() else None
            }
            for s in r['steps']
        ]
    }

def get_direction_func(params: DirectionsInput):
    places_result = gmaps.directions_result = gmaps.directions(
        origin=params.origin,
        destination=params.destination,
        mode=params.mode.value,
    )
    # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
    return [simplified_directions_dict(r) for r in places_result]

directions_tool_description = """\
Get directions between an origin point and a destination point.
"""

# get_direction_tool = Tool.from_function(
#     func = get_direction_func,
#     name='Directions',
#     description=directions_tool_description,
# )

class DirectionsTool(BaseTool):
    name = "Directions"
    description = directions_tool_description
    args_schema: Type[DirectionsInput] = DirectionsInput

    def _run(self, origin, destination, mode):
        print(mode.value)
        places_result = gmaps.directions_result = gmaps.directions(
            origin=origin,
            destination=destination,
            mode=mode.value,
        )
        # print(json.dumps(geocode_results,indent=2,ensure_ascii=False))
        return [simplified_directions_dict(r) for r in places_result]

    async def _arun(self, origin, destination, mode):
        return self._run(origin, destination, mode)

get_direction_tool = DirectionsTool()

print(get_direction_func(
    params=DirectionsInput(
        origin='Tower of London',
        destination='Backingum Palace',
        # origin={'latitude': '35.1796', 'longitude': '136.9166'},
        # destination={'latitude': '35.091708', 'longitude': '136.916568'},
        mode='transit'
    )
))

[{'arrival_time': '1:34\u202fAM', 'departure_time': '12:54\u202fAM', 'distance': '6.6 km', 'duration': '40 mins', 'steps': [{'distance': '0.4 km', 'duration': '5 mins', 'travel_mode': 'WALKING', 'instruction': 'Walk to The Tower Of London (Stop TA)', 'transit_details': None}, {'distance': '3.5 km', 'duration': '12 mins', 'travel_mode': 'TRANSIT', 'instruction': 'Bus towards Charing Cross', 'transit_details': {'arrival_stop': {'location': {'lat': 51.5107429, 'lng': -0.1204931}, 'name': 'Savoy Street (Stop U)'}, 'departure_stop': {'location': {'lat': 51.5095849, 'lng': -0.07552729999999999}, 'name': 'The Tower Of London (Stop TA)'}}}, {'distance': '2.0 km', 'duration': '9 mins', 'travel_mode': 'TRANSIT', 'instruction': 'Bus towards Westbourne Park', 'transit_details': {'arrival_stop': {'location': {'lat': 51.5066405, 'lng': -0.1428872}, 'name': 'Green Park'}, 'departure_stop': {'location': {'lat': 51.5107429, 'lng': -0.1204931}, 'name': 'Savoy Street (Stop U)'}}}, {'distance': '0.7 km', 

In [26]:
llm = ChatOpenAI(temperature=0)

In [43]:
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

In [44]:
tools = [
    get_current_location_tool,
    find_place_tool,
    get_direction_tool
]
agent = initialize_agent(
    tools, llm, agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory, agent_kwargs=agent_kwargs,
    verbose=True
)

In [40]:
input_text = 'Tower of Londonはどこにありますか？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `PlaceSearch` with `Tower of London`


[{'address': '4-chōme-23-9 Meieki, Nakamura Ward, Nagoya, Aichi 450-0002, Japan', 'location': {'lat': 35.168371, 'lng': 136.889019}, 'types': ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']}, {'address': 'London EC3N 4AB, UK', 'location': {'lat': 51.50811239999999, 'lng': -0.0759493}, 'types': ['establishment', 'landmark', 'point_of_interest', 'tourist_attraction']}]Tower of Londonは、イギリスのロンドンにあります。正確な住所は、London EC3N 4AB, UKです。

> Finished chain.


'Tower of Londonは、イギリスのロンドンにあります。正確な住所は、London EC3N 4AB, UKです。'

In [46]:
input_text = 'タワーオブロンドンはどこにありますか？'
# input_text = 'エスターテ南青山から愛知県名古屋市南区港東通１丁目２４まで徒歩何分ですか？'
agent.run(input_text)



> Entering new AgentExecutor chain...

Invoking: `PlaceSearch` with `タワーオブロンドン`


[{'address': '4-chōme-23-9 Meieki, Nakamura Ward, Nagoya, Aichi 450-0002, Japan', 'location': {'lat': 35.168371, 'lng': 136.889019}, 'types': ['bar', 'establishment', 'food', 'point_of_interest', 'restaurant']}]タワーオブロンドンは、日本の愛知県名古屋市中村区名駅4丁目23-9にあります。

> Finished chain.


'タワーオブロンドンは、日本の愛知県名古屋市中村区名駅4丁目23-9にあります。'

In [24]:
print(agent.memory.json(indent=2,ensure_ascii=False))

{
  "chat_memory": {
    "messages": [
      {
        "content": "名古屋駅に一番近いガストを教えて",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "名古屋駅に一番近いガストは、愛知県名古屋市中村区太閤通６丁目５９にあります。",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "名古屋駅からそのガストまで何分？",
        "additional_kwargs": {},
        "example": false
      },
      {
        "content": "申し訳ありませんが、現在の交通手段に関する情報が取得できませんでした。お手数ですが、別の方法でアクセス時間を調べていただくか、近くの交通機関の情報をご確認ください。",
        "additional_kwargs": {},
        "example": false
      }
    ]
  },
  "output_key": null,
  "input_key": null,
  "return_messages": true,
  "human_prefix": "Human",
  "ai_prefix": "AI",
  "memory_key": "memory"
}
